In [37]:
from pdf2image import convert_from_path

In [3]:
pages = convert_from_path(r'docs\patient_details\pd_1.pdf' , poppler_path = r'C:\poppler-24.02.0\Library\bin')

In [4]:
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1867x2000>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2000x1398>]

In [5]:
pages[0].show()

In [6]:
import pytesseract

In [7]:
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(pages[0], lang='eng')
print(text)

47/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Case of Emergency
m _ a _
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History

Chicken Pox (Varicella):

IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine



In [8]:
import numpy as np
import cv2
from PIL import Image

def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
    processed_image = cv2.adaptiveThreshold(
        resized,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 
        61,
        11
    )
    return processed_image

In [9]:
img = preprocess_image(pages[0])


Image.fromarray(img).show()

text = pytesseract.image_to_string(img, lang='eng')
print(text)

17/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight’
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Casc of Emergency
7 ee
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone

Genera! Medical History

a

a

a ea A CE i a

Chicken Pox (Varicella): Measies:

IMMUNE IMMUNE

Have you had the Hepatitis B vaccination?
No

List any Medical Problems (asthma, seizures, headaches}:

Migraine

CO
aa



**Extract Name**

In [15]:
import re

pattern = r'Patient Information(.*?)\(\d{3}\)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

[' Birth Date\nKathy Crawford May 6 1972\n']

In [16]:
matches[0].strip()

'Birth Date\nKathy Crawford May 6 1972'

In [18]:
match = matches[0].replace("Birth Date", "").strip()
match

'Kathy Crawford May 6 1972'

In [22]:
pattern = r'((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'

date_matches =  re.findall(pattern, text)
date_matches


[('May 6 1972', 'May')]

In [23]:
date = date_matches[0][0]
date

'May 6 1972'

In [24]:
match.replace(date,'').strip()

'Kathy Crawford'

In [25]:
def remove_noise_from_name(name):
    name = name.replace("Birth Date", "").strip()
    date_pattern =  r'((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    date_matches =  re.findall(date_pattern, name)

    if date_matches:
        date = date_matches[0][0]
        name = name.replace(date,'').strip()
        
    return name

In [26]:
name = 'Birth Date\nKathy Crawford May 6 1972'

name = remove_noise_from_name(name)
name

'Kathy Crawford'

**Extract Phone**

In [27]:
pattern = r'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

[(' Birth Date\nKathy Crawford May 6 1972\n', '(737) 988-0851')]

In [28]:
matches[0]

(' Birth Date\nKathy Crawford May 6 1972\n', '(737) 988-0851')

In [29]:
matches[0][1]

'(737) 988-0851'

**Extract Vaccine**

In [31]:
pattern = r'Have you had the Hepatitis B vaccination\?.*(Yes|No)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['No']

**Extract Medical Problems**

In [35]:
pattern = r'List any Medical Problems \(asthma, seizures, headaches\}:(.*)CO'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['\n\nMigraine\n\n']

In [38]:
matches[0].strip()

'Migraine'